In [403]:
import pandas as pd
import numpy as np
import os

In [404]:
path = "C:\\Users\\pl72955\\Documents\\kaggle\\"

In [405]:
DATA_FOLDER = path

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train_v2.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
train = transactions

In [406]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [407]:
train['date'] = pd.to_datetime(train['date'],format="%d.%m.%Y")

In [408]:
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['monthyear'] = train['month'].map(str)+train['year'].map(str)

In [409]:
train['shopitem'] = train['shop_id'].map(str)+train['item_id'].map(str)

In [410]:
#restrict to 2015
# train = train[train.year==2015]

In [411]:
train['key'] = train['shopitem'].map(str)+train['monthyear'].map(str)
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,monthyear,shopitem,key
0,2013-01-02,0,59,22154,999.00,1.0,1,2013,12013,5922154,592215412013
1,2013-01-03,0,25,2552,899.00,1.0,1,2013,12013,252552,25255212013
2,2013-01-05,0,25,2552,899.00,-1.0,1,2013,12013,252552,25255212013
3,2013-01-06,0,25,2554,1709.05,1.0,1,2013,12013,252554,25255412013
4,2013-01-15,0,25,2555,1099.00,1.0,1,2013,12013,252555,25255512013


In [412]:
monthgroup = train.groupby([ 'shopitem', 'monthyear',])['item_cnt_day'].sum()
monthgroupdf = pd.DataFrame(monthgroup)

In [413]:
monthgroupdf.reset_index(inplace=True)  

In [414]:
monthgroupdf['key'] = monthgroupdf['shopitem'].map(str)+monthgroupdf['monthyear'].map(str)
monthgroupdf.head()


,shopitem,monthyear,item_cnt_day,key
0,01000,12013,5.0,0100012013
1,01000,22013,4.0,0100022013
2,010004,22013,1.0,01000422013
3,01001,12013,2.0,0100112013
4,010012,12013,1.0,01001212013


In [415]:
monthgroupdf = monthgroupdf.merge(train, on='key', how='left').drop(['item_cnt_day_y','monthyear_y','shopitem_y'],axis=1)
monthgroupdf = monthgroupdf.drop_duplicates(subset='key')

In [416]:
monthgroupdf.head()

,shopitem_x,monthyear_x,item_cnt_day_x,key,date,date_block_num,shop_id,item_id,item_price,month,year
0,01000,12013,5.0,0100012013,2013-01-03,0,0,1000,58.0,1,2013
5,01000,22013,4.0,0100022013,2013-02-17,1,0,1000,58.0,2,2013
9,010004,22013,1.0,01000422013,2013-02-09,1,0,10004,64.0,2,2013
10,01001,12013,2.0,0100112013,2013-01-03,0,0,1001,58.0,1,2013
12,010012,12013,1.0,01001212013,2013-01-22,0,0,10012,76.0,1,2013


In [417]:
shop_items_unique0 = np.array(np.meshgrid(shops.shop_id.unique(), items.item_id.unique())).T.reshape(-1,2)
shop_items_unique = [''.join([str(y) for y in x]) for x in shop_items_unique0]


# shop_items_unique = monthgroupdf.shopitem_x.unique()

shop_items_unique_count = len(shop_items_unique)
month_year_unique = monthgroupdf.monthyear_x.unique()
month_year_unique_count =len(month_year_unique)

shop_items_expanded = [[shop_item]*month_year_unique_count for shop_item in shop_items_unique ]
shop_items_expanded = np.concatenate(shop_items_expanded)


month_year_expanded = [monthyear for monthyear in month_year_unique] * shop_items_unique_count
month_year_expanded = np.array(month_year_expanded)
zeroes_count = shop_items_unique_count * month_year_unique_count
item_count_expanded = np.array([0]*zeroes_count)
 

In [418]:
dataframe_dictionary = {'shopitem_x': shop_items_expanded, 'monthyear_x':  month_year_expanded, 'item_cnt_day_x':  item_count_expanded}


In [ ]:
zero_filler = pd.DataFrame(dataframe_dictionary)

In [419]:
zero_filler['key'] = zero_filler['shopitem_x'].map(str)+zero_filler['monthyear_x'].map(str)
zero_filler.head()

In [421]:
monthgroupdf_zeroed = monthgroupdf.merge(zero_filler, on=['key', 'item_cnt_day_x', 'monthyear_x', 'shopitem_x'], how='outer' , )
# zero_filler.key.head()
monthgroupdf_zeroed = monthgroupdf_zeroed.drop_duplicates(subset='key')
monthgroupdf_zeroed = monthgroupdf_zeroed[['key','item_cnt_day_x']]


MemoryError: cannot allocate memory for array

In [287]:
#impute missing values
#all store_item combinations should have a month_year row, and if it does not exist, create with a item_count_month of 0.


#clip training values from 0,20 as well
monthgroupdf_zeroed = monthgroupdf_zeroed.merge(train, on='key', how='left')
monthgroupdf_zeroed = monthgroupdf_zeroed.drop_duplicates(subset='key')

In [290]:
monthgroupdf_zeroed = monthgroupdf_zeroed.drop(['item_cnt_day'], axis=1)


In [291]:
monthgroupdf = monthgroupdf_zeroed

In [292]:
monthgroupdf = monthgroupdf.merge(items, on='item_id')

In [293]:
monthgroupdf = monthgroupdf.merge(item_categories, on='item_category_id')

In [294]:
monthgroupdf = monthgroupdf.merge(shops, on='shop_id')


In [304]:
train_expanded = monthgroupdf.rename(columns={'shopitem_x':'shopitem', 'monthyear_x': 'monthyear', 'item_cnt_day_x': 'item_cnt_month'})


In [305]:
train_expanded.head()

,key,item_cnt_month,date,date_block_num,shop_id,item_id,item_price,month,year,monthyear,shopitem,item_name,item_category_id,item_category_name,shop_name
0,0100012013,5.0,2013-01-03,0.0,0.0,1000.0,58.0,1.0,2013.0,12013,01000,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран"
1,0100022013,4.0,2013-02-17,1.0,0.0,1000.0,58.0,2.0,2013.0,22013,01000,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран"
2,0100112013,2.0,2013-01-03,0.0,0.0,1001.0,58.0,1.0,2013.0,12013,01001,"3D Action Puzzle ""Зомби"" Шахтер",67,Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран"
3,0100212013,2.0,2013-01-09,0.0,0.0,1002.0,58.0,1.0,2013.0,12013,01002,"3D Action Puzzle ""Техника"" Бомбардировщик",67,Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран"
4,0100312013,2.0,2013-01-09,0.0,0.0,1003.0,58.0,1.0,2013.0,12013,01003,"3D Action Puzzle ""Техника"" Вертолет",67,Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран"


In [306]:
print [x.count('-') for x in train_expanded.item_category_name.unique()]
def split_cat(input_str, pos=0):
    if (input_str.count('-')>=pos):
        return input_str.split('-')[pos]
    else:
        return ""
    
train_expanded['cat_0'] = train_expanded.item_category_name.apply(split_cat, args=(0,))
train_expanded['cat_1'] = train_expanded.item_category_name.apply(split_cat, args=(1,))
train_expanded['cat_2'] = train_expanded.item_category_name.apply(split_cat, args=(2,))

[1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [307]:
train_expanded = train_expanded.drop(['item_category_id', 'monthyear'],axis=1)
# train_expanded = train_expanded.drop(['shopitem'],axis=1)


In [308]:
train_expanded.head()

,key,item_cnt_month,date,date_block_num,shop_id,item_id,item_price,month,year,shopitem,item_name,item_category_name,shop_name,cat_0,cat_1,cat_2
0,0100012013,5.0,2013-01-03,0.0,0.0,1000.0,58.0,1.0,2013.0,01000,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,
1,0100022013,4.0,2013-02-17,1.0,0.0,1000.0,58.0,2.0,2013.0,01000,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,
2,0100112013,2.0,2013-01-03,0.0,0.0,1001.0,58.0,1.0,2013.0,01001,"3D Action Puzzle ""Зомби"" Шахтер",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,
3,0100212013,2.0,2013-01-09,0.0,0.0,1002.0,58.0,1.0,2013.0,01002,"3D Action Puzzle ""Техника"" Бомбардировщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,
4,0100312013,2.0,2013-01-09,0.0,0.0,1003.0,58.0,1.0,2013.0,01003,"3D Action Puzzle ""Техника"" Вертолет",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,


In [302]:
train_expanded_ohe = train_expanded
train_expanded_ohe = train_expanded_ohe.drop(['item_id','shop_id'],axis=1)
cols = ['cat_0', 'cat_1', 'cat_2', 'shop_name', 'item_category_name', 'month']
for col in cols:
    print (col)
    train_expanded_ohe = train_expanded_ohe.join(pd.get_dummies(train_expanded_ohe[col], prefix=col))
    train_expanded_ohe.drop([col],axis=1)

cat_0
cat_1
cat_2
shop_name
item_category_name


Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [328]:
train_expanded.sort_values(['shop_id','item_id','date'])

,key,item_cnt_month,date,date_block_num,shop_id,item_id,item_price,month,year,shopitem,item_name,item_category_name,shop_name,cat_0,cat_1,cat_2
1334,03022013,31.0,2013-02-21,1.0,0.0,30.0,265.00,2.0,2013.0,030,007: КООРДИНАТЫ «СКАЙФОЛЛ»,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",Кино,DVD,
2660,03122013,11.0,2013-02-15,1.0,0.0,31.0,434.00,2.0,2013.0,031,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран",Кино,Blu,Ray
1337,03212013,6.0,2013-01-03,0.0,0.0,32.0,221.00,1.0,2013.0,032,1+1,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",Кино,DVD,
1338,03222013,10.0,2013-02-25,1.0,0.0,32.0,221.00,2.0,2013.0,032,1+1,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",Кино,DVD,
2661,03312013,3.0,2013-01-03,0.0,0.0,33.0,347.00,1.0,2013.0,033,1+1 (BD),Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран",Кино,Blu,Ray
2662,03322013,3.0,2013-02-26,1.0,0.0,33.0,347.00,2.0,2013.0,033,1+1 (BD),Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран",Кино,Blu,Ray
1339,03512013,1.0,2013-01-31,0.0,0.0,35.0,247.00,1.0,2013.0,035,10 ЛЕТ СПУСТЯ,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",Кино,DVD,
1340,03522013,14.0,2013-02-01,1.0,0.0,35.0,247.00,2.0,2013.0,035,10 ЛЕТ СПУСТЯ,Кино - DVD,"!Якутск Орджоникидзе, 56 фран",Кино,DVD,
2663,03622013,1.0,2013-02-15,1.0,0.0,36.0,357.00,2.0,2013.0,036,10 ЛЕТ СПУСТЯ (BD),Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран",Кино,Blu,Ray
3585,04022013,1.0,2013-02-06,1.0,0.0,40.0,127.00,2.0,2013.0,040,100 Best classical melodies (mp3-CD) (Digipack),Музыка - MP3,"!Якутск Орджоникидзе, 56 фран",Музыка,MP3,


In [300]:
train_expanded.to_csv("to_data_r2.csv", headers=True, index=False)

In [79]:
X.head()

,key,date,item_price,month,year,item_name,item_category_name,shop_name,cat_0,cat_1,...,cat_1_ Сувениры,cat_1_ Сувениры (в навеску),"cat_1_ Сумки, Альбомы, Коврики д/мыши",cat_1_ Фигурки,cat_1_ Художественная литература,cat_1_ Цифра,cat_2_,cat_2_Ray,cat_2_Ray 3D,cat_2_Ray 4K
0,0100012013,2013-01-03,58.0,1,2013,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0100012013,2013-01-08,58.0,1,2013,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0100012013,2013-01-09,58.0,1,2013,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0100012013,2013-01-10,58.0,1,2013,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0100012013,2013-01-20,58.0,1,2013,"3D Action Puzzle ""Зомби"" Уборщик",Подарки - Развитие,"!Якутск Орджоникидзе, 56 фран",Подарки,Развитие,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [78]:
from sklearn.ensemble import RandomForestRegressor

X = train_expanded_ohe[(train_expanded_ohe.year <= 2015)]
y = train_expanded_ohe.item_cnt_month
X = X.drop(['item_cnt_month'],axis=1)

# clf = RandomForestRegressor()
# clf.fit(X, y)

In [100]:
mys = len(monthgroupdf.monthyear.unique()) 

In [101]:
monthgroupdf.item_cnt_day = monthgroupdf.item_cnt_day.divide(mys)

In [102]:
naive_results = pd.DataFrame(monthgroupdf.groupby('shopitem')['item_cnt_day'].sum())

In [103]:
naive_results.reset_index(inplace=True)  

In [104]:
naive_results.head()

,shopitem,item_cnt_day
0,01000,0.264706
1,010004,0.029412
2,01001,0.058824
3,010012,0.088235
4,01002,0.058824


In [80]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

In [316]:
test['shopitem'] = test['shop_id'].map(str)+test['item_id'].map(str)


# submit = test.merge(train_expanded.drop_duplicates(subset='shopitem'), on='shopitem', how='left').drop(['shop_id_y','item_id_y'],axis=1)
submit = test.merge(items, on='item_id')
submit = submit.merge(item_categories, on='item_category_id')
submit = submit.merge(shops, on='shop_id')

submit = submit.drop(['shopitem', 'item_category_id'],axis=1)
submit['cat_0'] = submit.item_category_name.apply(split_cat, args=(0,))
submit['cat_1'] = submit.item_category_name.apply(split_cat, args=(1,))
submit['cat_2'] = submit.item_category_name.apply(split_cat, args=(2,))
submit['key'] = submit['shop_id'].map(str)+submit['item_id'].map(str)+"112015"

In [317]:
submit.head()
submit = submit.rename(columns={'shop_id_x':'shop_id', 'item_id_x':'item_id'})


In [318]:
submit.head()

,ID,shop_id,item_id,item_name,item_category_name,shop_name,cat_0,cat_1,cat_2,key
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3,,55037112015
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3,,55233112015
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3,,54872112015
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3,,55002112015
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",Игры - PS3,"Вологда ТРЦ ""Мармелад""",Игры,PS3,,55814112015


In [133]:
print sorted(train_expanded.columns)
print sorted(submit.columns)

['cat_0', 'cat_1', 'cat_2', 'date', 'item_category_name', 'item_cnt_month', 'item_id', 'item_name', 'item_price', 'key', 'month', 'shop_id', 'shop_name', 'shopitem', 'year']
['ID', 'cat_0', 'cat_1', 'cat_2', 'date', 'item_category_name', 'item_cnt_month', 'item_id', 'item_name', 'item_price', 'key', 'month', 'shop_id', 'shop_name', 'year']


In [320]:
submit.to_csv("dr_test2.csv", header=True, index=False)

In [109]:
total_rows = len(submit.item_cnt_day)
non_nan_rows = submit.item_cnt_day.count()
nan_rows =  total_rows - non_nan_rows
item_mean = submit.item_cnt_day.mean()
print (count_nan)
print (len(submit.item_cnt_day))
print (item_mean)

98068
214200
0.430609134434


In [110]:
submit['item_cnt_day'] = submit['item_cnt_day'].fillna(item_mean/2.0)

In [111]:
submit = submit[['ID','item_cnt_day']]

submit.item_cnt_day = submit.item_cnt_day.clip(lower=0,upper=20)
submit = submit.rename(columns={'item_cnt_day':'item_cnt_month'})

In [112]:
submit.head()

,ID,item_cnt_month
0,0,0.382353
1,1,0.215305
2,2,0.294118
3,3,0.029412
4,4,0.029412


In [113]:
submit.sort_values(by=['item_cnt_month'], ascending=False).head()

,ID,item_cnt_month
37350,37350,20.0
39758,39758,20.0
46174,46174,20.0
46989,46989,20.0
157330,157330,20.0


In [114]:
submit.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.311344
std,61834.358168,0.743125
min,0.000000,0.000000
25%,53549.750000,0.117647
50%,107099.500000,0.215305
75%,160649.250000,0.215305
max,214199.000000,20.000000


In [115]:
submit.to_csv('submit3.csv' , index=False, header=True)